In [1]:
import torch
# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

print("Device being used: %s" %device)

There are 1 GPU(s) available.
We will use the GPU: Tesla V100-SXM2-16GB
Device being used: cuda


In [2]:
path = '/scratch/kap9623/Final'
print(path)

/scratch/kap9623/Final


In [3]:
%%capture
!pip install simpletransformers

In [4]:
#retrieving paths for ann data

import glob
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from simpletransformers.ner import NERModel,NERArgs
# import wandb

ann_orig = glob.glob(path + '/ann_ref/*.ann' )
print(len(ann_orig))

389


In [5]:
#setting hyperparameters to default for all models
args = NERArgs()
args.learning_rate = 1e-4
args.scheduler = 'constant_schedule'
args.overwrite_output_dir = True
args.manual_seed = 42
args.train_batch_size = 32
args.eval_batch_size = 32
args.num_train_epochs = 10

# **Task 1: Name Entry Recognition for Medication Extraction**

# MODEL: Baseline BERT

## Note-level Split

### Reformatting from BRAT to ConLL

In [6]:
outfile = path + '/BaseBERT/output_note_lvl.txt'
with open(outfile, 'w') as output_file:
  out_list = []
  for ann_file in ann_orig:
    with open(ann_file, 'r') as ann_in:
      txt_file = path + f'/ann_ref/{ann_file[-10:-4]}.txt'
      with open(txt_file, 'r') as txt_in:
        start_end = []
        
        #reading each line of .ann file and extracting start and end indices 
        for line in ann_in:
          entry = line.split()
          start_end.append([int(entry[2]), int(entry[3])])

        #reading each line of .txt file and extracting drug names using list of indices
        text_tokens = txt_in.read().split()
        drug_names = []
        for indicies in start_end:
          start = indicies[0]
          end = indicies[1]

          with open(txt_file, 'r') as txt_in:
            drug_names.append(txt_in.read()[indicies[0]:indicies[1]])
        
        #appending "O" or "DRUG" label to each txt token with relevant spacers
        for token in text_tokens:
          token_alnum = ''.join(ch for ch in token if ch.isalnum())
          if token_alnum in drug_names:
            if token == text_tokens[0]:
              out_list.append(f'{ann_file[-10:-4]}\t') #NOTE LEVEL
            else:
              out_list.append(f'{np.nan}\t')
            out_list.append(f'{token}\tDRUG\n')
          else:
            if token == text_tokens[0]:
              out_list.append(f'{ann_file[-10:-4]}\t') #NOTE LEVEL
            else:
              out_list.append(f'{np.nan}\t')
            out_list.append(f'{token}\tO\n')
        out_list.append('\n')
  out_string = ''.join(out_list)
  output_file.write(out_string)

### Reading ConLL output as dataframe

In [7]:
data = pd.read_csv(path + '/BaseBERT/output_note_lvl.txt', sep='\t', lineterminator='\n', names = ['Note', 'Token', 'Label'])
data = data.fillna(method = 'ffill')

In [8]:
#adding token count for each token by note
token_count = []
for note in data['Note'].unique():
  df = data.loc[data['Note']== note]
  count = 0
  for token in df['Token']:
    token_count.append(count)
    count += 1

data['Token Count'] = token_count
data.sort_values(by = ['Note', 'Token Count'], inplace=True)
data.reset_index(drop = True, inplace = True)

In [9]:
#label encoding Note column
le = LabelEncoder()
note_en = le.fit_transform(data['Note'])
data['Note'] = note_en

In [10]:
#renaming columns to match model requirements
data.rename(columns = {'Note':'sentence_id', 'Token':'words', 'Label':'labels'}, inplace=True)

### Model Building

In [11]:
#splitting data into featues and labels
X = data[['sentence_id','words']]
y = data['labels']

#splitting into train, val, test (70-10-20)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, shuffle=False)
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size = 0.125, shuffle=False)

#building up train data and test data
train_data = pd.DataFrame({"sentence_id":X_train["sentence_id"],"words":X_train["words"],"labels":y_train})
valid_data = pd.DataFrame({"sentence_id":X_valid["sentence_id"],"words":X_valid["words"],"labels":y_valid})
test_data = pd.DataFrame({"sentence_id":X_test["sentence_id"],"words":X_test["words"],"labels":y_test})

In [12]:
#checking unique token labels
label = data["labels"].unique().tolist()
label

['O', 'DRUG']

In [13]:
#instantiating model
model_base_note = NERModel('bert', 'bert-base-cased',labels=label,args =args, use_cuda = True)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForTokenClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cas

In [14]:
#training model on train data and tuning hyperparameters on valid data
model_base_note.train_model(train_data,eval_data = valid_data)

  0%|          | 0/1 [00:00<?, ?it/s]

/home/kap9623/.local/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

Running Epoch 0 of 10:   0%|          | 0/9 [00:00<?, ?it/s]

/ext3/miniconda3/lib/python3.9/site-packages/torch/optim/lr_scheduler.py:129: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "


...

Running Epoch 6 of 10:   0%|          | 0/9 [00:00<?, ?it/s]

Running Epoch 7 of 10:   0%|          | 0/9 [00:00<?, ?it/s]

Running Epoch 8 of 10:   0%|          | 0/9 [00:00<?, ?it/s]

Running Epoch 9 of 10:   0%|          | 0/9 [00:00<?, ?it/s]

(90, 0.04403139898147654)

In [15]:
# #reinstantiating NERModel using best epoch from training
# model_base_note = NERModel('bert', 'outputs/best_model')

In [16]:
#evaluating on test data
base_note_result, base_note_outputs, base_note_preds = model_base_note.eval_model(test_data)

  0%|          | 0/1 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/3 [00:00<?, ?it/s]

/home/kap9623/.local/lib/python3.9/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: DRUG seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))


In [17]:
print('Model: BaseBERT')
print('Evaluation metrics (Note-level split):')
base_note_result

Model: BaseBERT
Evaluation metrics (Note-level split):


{'eval_loss': 0.017991996370255947,
 'precision': 1.0,
 'recall': 0.5531914893617021,
 'f1_score': 0.7123287671232876}

In [19]:
model_base_note.predict(['As a result of this, I think it is reasonable for us in addition to having her on atenolol to stop the hydrochlorothiazide, put her on ramipril and a nitrate.'])

  0%|          | 0/1 [00:00<?, ?it/s]

Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

([[{'As': 'O'},
   {'a': 'O'},
   {'result': 'O'},
   {'of': 'O'},
   {'this,': 'O'},
   {'I': 'O'},
   {'think': 'O'},
   {'it': 'O'},
   {'is': 'O'},
   {'reasonable': 'O'},
   {'for': 'O'},
   {'us': 'O'},
   {'in': 'O'},
   {'addition': 'O'},
   {'to': 'O'},
   {'having': 'O'},
   {'her': 'O'},
   {'on': 'O'},
   {'atenolol': 'DRUG'},
   {'to': 'O'},
   {'stop': 'O'},
   {'the': 'O'},
   {'hydrochlorothiazide,': 'DRUG'},
   {'put': 'O'},
   {'her': 'O'},
   {'on': 'O'},
   {'ramipril': 'DRUG'},
   {'and': 'O'},
   {'a': 'O'},
   {'nitrate.': 'DRUG'}]],
 [[{'As': [[4.824, -4.32]]},
   {'a': [[4.918, -4.293]]},
   {'result': [[4.46, -3.764]]},
   {'of': [[4.324, -4.11]]},
   {'this,': [[4.453, -3.928], [4.42, -4.004]]},
   {'I': [[4.55, -4.016]]},
   {'think': [[4.52, -3.895]]},
   {'it': [[4.465, -3.98]]},
   {'is': [[4.777, -4.203]]},
   {'reasonable': [[4.67, -4.273]]},
   {'for': [[4.688, -4.316]]},
   {'us': [[4.77, -4.35]]},
   {'in': [[4.953, -4.18]]},
   {'addition': [[4.703,

## Patient-level Split

### Reformatting from BRAT to ConLL

In [19]:
outfile = path + '/BaseBERT/output_pt_lvl.txt'
with open(outfile, 'w') as output_file:
  out_list = []
  for ann_file in ann_orig:
    with open(ann_file, 'r') as ann_in:
      txt_file = path + f'/ann_ref/{ann_file[-10:-4]}.txt'
      with open(txt_file, 'r') as txt_in:
        start_end = []
        
        #reading each line of .ann file and extracting start and end indices 
        for line in ann_in:
          entry = line.split()
          start_end.append([int(entry[2]), int(entry[3])])

        #reading each line of .txt file and extracting drug names using list of indices
        text_tokens = txt_in.read().split()
        drug_names = []
        for indicies in start_end:
          start = indicies[0]
          end = indicies[1]

          with open(txt_file, 'r') as txt_in:
            drug_names.append(txt_in.read()[indicies[0]:indicies[1]])
        
        #appending "O" or "DRUG" label to each txt token with relevant spacers
        for token in text_tokens:
          token_alnum = ''.join(ch for ch in token if ch.isalnum())
          if token_alnum in drug_names:
            if token == text_tokens[0]:
              out_list.append(f'{ann_file[-10:-7]}\t') #PATIENT LEVEL
            else:
              out_list.append(f'{np.nan}\t')
            out_list.append(f'{token}\tDRUG\n')
          else:
            if token == text_tokens[0]:
              out_list.append(f'{ann_file[-10:-7]}\t') #PATIENT LEVEL
            else:
              out_list.append(f'{np.nan}\t')
            out_list.append(f'{token}\tO\n')
        out_list.append('\n')
  out_string = ''.join(out_list)
  output_file.write(out_string)

### Reading ConLL output as dataframe

In [20]:
data = pd.read_csv(path + '/BaseBERT/output_pt_lvl.txt', sep='\t', lineterminator='\n', names = ['Patient', 'Token', 'Label'])
data = data.fillna(method = 'ffill')

In [21]:
#adding token count for each token by note
token_count = []
for note in data['Patient'].unique():
  df = data.loc[data['Patient']== note]
  count = 0
  for token in df['Token']:
    token_count.append(count)
    count += 1

data['Token Count'] = token_count
data.sort_values(by = ['Patient', 'Token Count'], inplace=True)
data.reset_index(drop = True, inplace = True)

In [22]:
#label encoding Note column
le = LabelEncoder()
note_en = le.fit_transform(data['Patient'])
data['Patient'] = note_en

In [23]:
#renaming columns to match model requirements
data.rename(columns = {'Patient':'sentence_id', 'Token':'words', 'Label':'labels'}, inplace=True)

### Model Building

In [24]:
#splitting data into featues and labels
X = data[['sentence_id','words']]
y = data['labels']

#splitting into train, val, test (70-10-20)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, shuffle=False)
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size = 0.125, shuffle=False)

#building up train data and test data
train_data = pd.DataFrame({"sentence_id":X_train["sentence_id"],"words":X_train["words"],"labels":y_train})
valid_data = pd.DataFrame({"sentence_id":X_valid["sentence_id"],"words":X_valid["words"],"labels":y_valid})
test_data = pd.DataFrame({"sentence_id":X_test["sentence_id"],"words":X_test["words"],"labels":y_test})

In [25]:
#checking unique token labels
label = data["labels"].unique().tolist()
label

['O', 'DRUG']

In [26]:
#instantiating model
model_base_pt = NERModel('bert', 'bert-base-cased',labels=label,args =args, use_cuda = True)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForTokenClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cas

In [27]:
#training model on train data and tuning hyperparameters on valid data
model_base_pt.train_model(train_data,eval_data = valid_data)

  0%|          | 0/1 [00:00<?, ?it/s]

/home/kap9623/.local/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

Running Epoch 0 of 10:   0%|          | 0/6 [00:00<?, ?it/s]

/ext3/miniconda3/lib/python3.9/site-packages/torch/optim/lr_scheduler.py:129: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "


...

Running Epoch 6 of 10:   0%|          | 0/6 [00:00<?, ?it/s]

Running Epoch 7 of 10:   0%|          | 0/6 [00:00<?, ?it/s]

Running Epoch 8 of 10:   0%|          | 0/6 [00:00<?, ?it/s]

Running Epoch 9 of 10:   0%|          | 0/6 [00:00<?, ?it/s]

(60, 0.07597993144906165)

In [28]:
# #reinstantiating NERModel using best epoch from training
# model_base_pt = NERModel('bert', 'outputs/best_model')

In [29]:
#evaluating on test data
base_pt_result, base_pt_outputs, base_pt_preds = model_base_pt.eval_model(test_data)

  0%|          | 0/1 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/2 [00:00<?, ?it/s]

/home/kap9623/.local/lib/python3.9/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: DRUG seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))


In [30]:
print('Model: BaseBERT')
print('Evaluation metrics (Patient-level split):')
base_pt_result

Model: BaseBERT
Evaluation metrics (Patient-level split):


{'eval_loss': 0.09883877262473106,
 'precision': 0.9,
 'recall': 0.39705882352941174,
 'f1_score': 0.5510204081632653}

In [31]:
model_base_pt.predict(['As a result of this, I think it is reasonable for us in addition to having her on atenolol to stop the hydrochlorothiazide, put her on ramipril and a nitrate.'])

  0%|          | 0/1 [00:00<?, ?it/s]

Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

([[{'As': 'O'},
   {'a': 'O'},
   {'result': 'O'},
   {'of': 'O'},
   {'this,': 'O'},
   {'I': 'O'},
   {'think': 'O'},
   {'it': 'O'},
   {'is': 'O'},
   {'reasonable': 'O'},
   {'for': 'O'},
   {'us': 'O'},
   {'in': 'O'},
   {'addition': 'O'},
   {'to': 'O'},
   {'having': 'O'},
   {'her': 'O'},
   {'on': 'O'},
   {'atenolol': 'DRUG'},
   {'to': 'O'},
   {'stop': 'O'},
   {'the': 'O'},
   {'hydrochlorothiazide,': 'DRUG'},
   {'put': 'O'},
   {'her': 'O'},
   {'on': 'O'},
   {'ramipril': 'DRUG'},
   {'and': 'O'},
   {'a': 'O'},
   {'nitrate.': 'DRUG'}]],
 [[{'As': [[4.32, -3.94]]},
   {'a': [[4.492, -3.936]]},
   {'result': [[4.375, -3.844]]},
   {'of': [[4.285, -3.91]]},
   {'this,': [[4.348, -3.877], [4.18, -3.87]]},
   {'I': [[4.336, -3.922]]},
   {'think': [[4.37, -3.975]]},
   {'it': [[4.363, -3.955]]},
   {'is': [[4.35, -3.984]]},
   {'reasonable': [[4.33, -4.062]]},
   {'for': [[4.367, -4.113]]},
   {'us': [[4.23, -3.96]]},
   {'in': [[4.3, -3.99]]},
   {'addition': [[4.305, -

# MODEL: BioBERT

## Note-level Split


### Reformatting from BRAT to ConLL

In [32]:
outfile = path + '/BioBERT/output_note_lvl.txt'
with open(outfile, 'w') as output_file:
  out_list = []
  for ann_file in ann_orig:
    with open(ann_file, 'r') as ann_in:
      txt_file = path + f'/ann_ref/{ann_file[-10:-4]}.txt'
      with open(txt_file, 'r') as txt_in:
        start_end = []
        
        #reading each line of .ann file and extracting start and end indices 
        for line in ann_in:
          entry = line.split()
          start_end.append([int(entry[2]), int(entry[3])])

        #reading each line of .txt file and extracting drug names using list of indices
        text_tokens = txt_in.read().split()
        drug_names = []
        for indicies in start_end:
          start = indicies[0]
          end = indicies[1]

          with open(txt_file, 'r') as txt_in:
            drug_names.append(txt_in.read()[indicies[0]:indicies[1]])
        
        #appending "O" or "DRUG" label to each txt token with relevant spacers
        for token in text_tokens:
          token_alnum = ''.join(ch for ch in token if ch.isalnum())
          if token_alnum in drug_names:
            if token == text_tokens[0]:
              out_list.append(f'{ann_file[-10:-4]}\t') #NOTE LEVEL
            else:
              out_list.append(f'{np.nan}\t')
            out_list.append(f'{token}\tDRUG\n')
          else:
            if token == text_tokens[0]:
              out_list.append(f'{ann_file[-10:-4]}\t') #NOTE LEVEL
            else:
              out_list.append(f'{np.nan}\t')
            out_list.append(f'{token}\tO\n')
        out_list.append('\n')
  out_string = ''.join(out_list)
  output_file.write(out_string)

### Reading ConLL output as dataframe

In [33]:
data = pd.read_csv(path + '/BioBERT/output_note_lvl.txt', sep='\t', lineterminator='\n', names = ['Note', 'Token', 'Label'])
data = data.fillna(method = 'ffill')

In [34]:
#adding token count for each token by note
token_count = []
for note in data['Note'].unique():
  df = data.loc[data['Note']== note]
  count = 0
  for token in df['Token']:
    token_count.append(count)
    count += 1

data['Token Count'] = token_count
data.sort_values(by = ['Note', 'Token Count'], inplace=True)
data.reset_index(drop = True, inplace = True)

In [35]:
#label encoding Note column
le = LabelEncoder()
note_en = le.fit_transform(data['Note'])
data['Note'] = note_en

In [36]:
#renaming columns to match model requirements
data.rename(columns = {'Note':'sentence_id', 'Token':'words', 'Label':'labels'}, inplace=True)

### Model Building

In [37]:
#splitting data into featues and labels
X = data[['sentence_id','words']]
y = data['labels']

#splitting into train, val, test (70-10-20)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, shuffle=False)
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size = 0.125, shuffle=False)

#building up train data and test data
train_data = pd.DataFrame({"sentence_id":X_train["sentence_id"],"words":X_train["words"],"labels":y_train})
valid_data = pd.DataFrame({"sentence_id":X_valid["sentence_id"],"words":X_valid["words"],"labels":y_valid})
test_data = pd.DataFrame({"sentence_id":X_test["sentence_id"],"words":X_test["words"],"labels":y_test})

In [38]:
#checking unique token labels
label = data["labels"].unique().tolist()
label

['O', 'DRUG']

In [39]:
#instantiating model
model_bio_note = NERModel('bert', 'dmis-lab/biobert-base-cased-v1.2',labels=label,args =args, use_cuda = True)

Some weights of the model checkpoint at dmis-lab/biobert-base-cased-v1.2 were not used when initializing BertForTokenClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initi

In [40]:
#training model on train data and tuning hyperparameters on valid data
model_bio_note.train_model(train_data,eval_data = valid_data)

  0%|          | 0/1 [00:00<?, ?it/s]

/home/kap9623/.local/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

Running Epoch 0 of 10:   0%|          | 0/9 [00:00<?, ?it/s]

/ext3/miniconda3/lib/python3.9/site-packages/torch/optim/lr_scheduler.py:129: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "


...

Running Epoch 6 of 10:   0%|          | 0/9 [00:00<?, ?it/s]

Running Epoch 7 of 10:   0%|          | 0/9 [00:00<?, ?it/s]

Running Epoch 8 of 10:   0%|          | 0/9 [00:00<?, ?it/s]

Running Epoch 9 of 10:   0%|          | 0/9 [00:00<?, ?it/s]

(90, 0.042991360402488174)

In [41]:
# #reinstantiating NERModel using best epoch from training
# model_bio_note = NERModel('bert', 'outputs/best_model')

In [42]:
#evaluating on test data
mbn_result, mbn_outputs, mbn_preds = model_bio_note.eval_model(test_data)

  0%|          | 0/1 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/3 [00:00<?, ?it/s]

/home/kap9623/.local/lib/python3.9/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: DRUG seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))


In [43]:
print('Model: BioBERT')
print('Evaluation metrics (Note-level split):')
mbn_result

Model: BioBERT
Evaluation metrics (Note-level split):


{'eval_loss': 0.0115911103785038,
 'precision': 1.0,
 'recall': 0.723404255319149,
 'f1_score': 0.8395061728395062}

In [44]:
model_bio_note.predict(['As a result of this, I think it is reasonable for us in addition to having her on atenolol to stop the hydrochlorothiazide, put her on ramipril and a nitrate.'])

  0%|          | 0/1 [00:00<?, ?it/s]

Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

([[{'As': 'O'},
   {'a': 'O'},
   {'result': 'O'},
   {'of': 'O'},
   {'this,': 'O'},
   {'I': 'O'},
   {'think': 'O'},
   {'it': 'O'},
   {'is': 'O'},
   {'reasonable': 'O'},
   {'for': 'O'},
   {'us': 'O'},
   {'in': 'O'},
   {'addition': 'O'},
   {'to': 'O'},
   {'having': 'O'},
   {'her': 'O'},
   {'on': 'O'},
   {'atenolol': 'DRUG'},
   {'to': 'O'},
   {'stop': 'O'},
   {'the': 'O'},
   {'hydrochlorothiazide,': 'DRUG'},
   {'put': 'O'},
   {'her': 'O'},
   {'on': 'O'},
   {'ramipril': 'DRUG'},
   {'and': 'O'},
   {'a': 'O'},
   {'nitrate.': 'DRUG'}]],
 [[{'As': [[5.355, -4.883]]},
   {'a': [[5.49, -4.973]]},
   {'result': [[5.57, -4.953]]},
   {'of': [[5.43, -4.97]]},
   {'this,': [[5.273, -4.85], [5.04, -4.695]]},
   {'I': [[5.336, -4.63]]},
   {'think': [[5.383, -4.715]]},
   {'it': [[5.484, -4.695]]},
   {'is': [[5.402, -4.812]]},
   {'reasonable': [[5.31, -4.797]]},
   {'for': [[5.527, -4.773]]},
   {'us': [[5.355, -4.777]]},
   {'in': [[5.527, -4.742]]},
   {'addition': [[5.2

## Patient-level Split

### Reformatting from BRAT to ConLL

In [45]:
outfile = path + '/BioBERT/output_pt_lvl.txt'
with open(outfile, 'w') as output_file:
  out_list = []
  for ann_file in ann_orig:
    with open(ann_file, 'r') as ann_in:
      txt_file = path + f'/ann_ref/{ann_file[-10:-4]}.txt'
      with open(txt_file, 'r') as txt_in:
        start_end = []
        
        #reading each line of .ann file and extracting start and end indices 
        for line in ann_in:
          entry = line.split()
          start_end.append([int(entry[2]), int(entry[3])])

        #reading each line of .txt file and extracting drug names using list of indices
        text_tokens = txt_in.read().split()
        drug_names = []
        for indicies in start_end:
          start = indicies[0]
          end = indicies[1]

          with open(txt_file, 'r') as txt_in:
            drug_names.append(txt_in.read()[indicies[0]:indicies[1]])
        
        #appending "O" or "DRUG" label to each txt token with relevant spacers
        for token in text_tokens:
          token_alnum = ''.join(ch for ch in token if ch.isalnum())
          if token_alnum in drug_names:
            if token == text_tokens[0]:
              out_list.append(f'{ann_file[-10:-7]}\t') #PATIENT LEVEL
            else:
              out_list.append(f'{np.nan}\t')
            out_list.append(f'{token}\tDRUG\n')
          else:
            if token == text_tokens[0]:
              out_list.append(f'{ann_file[-10:-7]}\t') #PATIENT LEVEL
            else:
              out_list.append(f'{np.nan}\t')
            out_list.append(f'{token}\tO\n')
        out_list.append('\n')
  out_string = ''.join(out_list)
  output_file.write(out_string)

### Reading ConLL output as dataframe

In [46]:
data = pd.read_csv(path + '/BioBERT/output_pt_lvl.txt', sep='\t', lineterminator='\n', names = ['Patient', 'Token', 'Label'])
data = data.fillna(method = 'ffill')

In [47]:
#adding token count for each token by note
token_count = []
for note in data['Patient'].unique():
  df = data.loc[data['Patient']== note]
  count = 0
  for token in df['Token']:
    token_count.append(count)
    count += 1

data['Token Count'] = token_count
data.sort_values(by = ['Patient', 'Token Count'], inplace=True)
data.reset_index(drop = True, inplace = True)

In [48]:
#label encoding Note column
le = LabelEncoder()
note_en = le.fit_transform(data['Patient'])
data['Patient'] = note_en

In [49]:
#renaming columns to match model requirements
data.rename(columns = {'Patient':'sentence_id', 'Token':'words', 'Label':'labels'}, inplace=True)

### Model Building

In [50]:
#splitting data into featues and labels
X = data[['sentence_id','words']]
y = data['labels']

#splitting into train, val, test (70-10-20)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, shuffle=False)
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size = 0.125, shuffle=False)

#building up train data and test data
train_data = pd.DataFrame({"sentence_id":X_train["sentence_id"],"words":X_train["words"],"labels":y_train})
valid_data = pd.DataFrame({"sentence_id":X_valid["sentence_id"],"words":X_valid["words"],"labels":y_valid})
test_data = pd.DataFrame({"sentence_id":X_test["sentence_id"],"words":X_test["words"],"labels":y_test})

In [51]:
#checking unique token labels
label = data["labels"].unique().tolist()
label

['O', 'DRUG']

In [52]:
#instantiating model
model_bio_pt = NERModel('bert', 'dmis-lab/biobert-base-cased-v1.2',labels=label,args =args, use_cuda = True)

Some weights of the model checkpoint at dmis-lab/biobert-base-cased-v1.2 were not used when initializing BertForTokenClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initi

In [53]:
#training model on train data and tuning hyperparameters on valid data
model_bio_pt.train_model(train_data,eval_data = valid_data)

  0%|          | 0/1 [00:00<?, ?it/s]

/home/kap9623/.local/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

Running Epoch 0 of 10:   0%|          | 0/6 [00:00<?, ?it/s]

/ext3/miniconda3/lib/python3.9/site-packages/torch/optim/lr_scheduler.py:129: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "


...

Running Epoch 6 of 10:   0%|          | 0/6 [00:00<?, ?it/s]

Running Epoch 7 of 10:   0%|          | 0/6 [00:00<?, ?it/s]

Running Epoch 8 of 10:   0%|          | 0/6 [00:00<?, ?it/s]

Running Epoch 9 of 10:   0%|          | 0/6 [00:00<?, ?it/s]

(60, 0.08390128941003544)

In [54]:
# #reinstantiating NERModel using best epoch from training
# model_bio_note = NERModel('bert', 'outputs/best_model')

In [55]:
#evaluating on test data
mbp_result, mbp_outputs, mbp_preds = model_bio_pt.eval_model(test_data)

  0%|          | 0/1 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/2 [00:00<?, ?it/s]

/home/kap9623/.local/lib/python3.9/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: DRUG seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))


In [56]:
print('Model: BioBERT')
print('Evaluation metrics (Patient-level split):')
mbp_result

Model: BioBERT
Evaluation metrics (Patient-level split):


{'eval_loss': 0.057154745794832706,
 'precision': 0.8888888888888888,
 'recall': 0.5882352941176471,
 'f1_score': 0.7079646017699115}

In [57]:
model_bio_pt.predict(['As a result of this, I think it is reasonable for us in addition to having her on atenolol to stop the hydrochlorothiazide, put her on ramipril and a nitrate.'])

  0%|          | 0/1 [00:00<?, ?it/s]

Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

([[{'As': 'O'},
   {'a': 'O'},
   {'result': 'O'},
   {'of': 'O'},
   {'this,': 'O'},
   {'I': 'O'},
   {'think': 'O'},
   {'it': 'O'},
   {'is': 'O'},
   {'reasonable': 'O'},
   {'for': 'O'},
   {'us': 'O'},
   {'in': 'O'},
   {'addition': 'O'},
   {'to': 'O'},
   {'having': 'O'},
   {'her': 'O'},
   {'on': 'O'},
   {'atenolol': 'DRUG'},
   {'to': 'O'},
   {'stop': 'O'},
   {'the': 'O'},
   {'hydrochlorothiazide,': 'DRUG'},
   {'put': 'O'},
   {'her': 'O'},
   {'on': 'O'},
   {'ramipril': 'DRUG'},
   {'and': 'O'},
   {'a': 'O'},
   {'nitrate.': 'O'}]],
 [[{'As': [[5.13, -4.277]]},
   {'a': [[5.03, -4.223]]},
   {'result': [[5.16, -4.266]]},
   {'of': [[4.992, -4.188]]},
   {'this,': [[5.05, -4.332], [4.93, -4.21]]},
   {'I': [[5.11, -4.188]]},
   {'think': [[5.07, -4.22]]},
   {'it': [[5.055, -4.195]]},
   {'is': [[4.977, -4.316]]},
   {'reasonable': [[5.12, -4.363]]},
   {'for': [[5.168, -4.383]]},
   {'us': [[5.12, -4.348]]},
   {'in': [[5.1, -4.34]]},
   {'addition': [[5.098, -4.37

# MODEL: SciBERT

## Note-level Split

### Reformatting from BRAT to ConLL

In [58]:
outfile = path + '/SciBERT/output_note_lvl.txt'
with open(outfile, 'w') as output_file:
  out_list = []
  for ann_file in ann_orig:
    with open(ann_file, 'r') as ann_in:
      txt_file = path + f'/ann_ref/{ann_file[-10:-4]}.txt'
      with open(txt_file, 'r') as txt_in:
        start_end = []
        
        #reading each line of .ann file and extracting start and end indices 
        for line in ann_in:
          entry = line.split()
          start_end.append([int(entry[2]), int(entry[3])])

        #reading each line of .txt file and extracting drug names using list of indices
        text_tokens = txt_in.read().split()
        drug_names = []
        for indicies in start_end:
          start = indicies[0]
          end = indicies[1]

          with open(txt_file, 'r') as txt_in:
            drug_names.append(txt_in.read()[indicies[0]:indicies[1]])
        
        #appending "O" or "DRUG" label to each txt token with relevant spacers
        for token in text_tokens:
          token_alnum = ''.join(ch for ch in token if ch.isalnum())
          if token_alnum in drug_names:
            if token == text_tokens[0]:
              out_list.append(f'{ann_file[-10:-4]}\t') #NOTE LEVEL
            else:
              out_list.append(f'{np.nan}\t')
            out_list.append(f'{token}\tDRUG\n')
          else:
            if token == text_tokens[0]:
              out_list.append(f'{ann_file[-10:-4]}\t') #NOTE LEVEL
            else:
              out_list.append(f'{np.nan}\t')
            out_list.append(f'{token}\tO\n')
        out_list.append('\n')
  out_string = ''.join(out_list)
  output_file.write(out_string)

### Reading ConLL output as dataframe

In [59]:
data = pd.read_csv(path + '/SciBERT/output_note_lvl.txt', sep='\t', lineterminator='\n', names = ['Note', 'Token', 'Label'])
data = data.fillna(method = 'ffill')

In [60]:
#adding token count for each token by note
token_count = []
for note in data['Note'].unique():
  df = data.loc[data['Note']== note]
  count = 0
  for token in df['Token']:
    token_count.append(count)
    count += 1

data['Token Count'] = token_count
data.sort_values(by = ['Note', 'Token Count'], inplace=True)
data.reset_index(drop = True, inplace = True)

In [61]:
#label encoding Note column
le = LabelEncoder()
note_en = le.fit_transform(data['Note'])
data['Note'] = note_en

In [62]:
#renaming columns to match model requirements
data.rename(columns = {'Note':'sentence_id', 'Token':'words', 'Label':'labels'}, inplace=True)

### Model Building

In [63]:
#splitting data into featues and labels
X = data[['sentence_id','words']]
y = data['labels']

#splitting into train, val, test (70-10-20)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, shuffle=False)
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size = 0.125, shuffle=False)

#building up train data and test data
train_data = pd.DataFrame({"sentence_id":X_train["sentence_id"],"words":X_train["words"],"labels":y_train})
valid_data = pd.DataFrame({"sentence_id":X_valid["sentence_id"],"words":X_valid["words"],"labels":y_valid})
test_data = pd.DataFrame({"sentence_id":X_test["sentence_id"],"words":X_test["words"],"labels":y_test})

In [64]:
#checking unique token labels
label = data["labels"].unique().tolist()
label

['O', 'DRUG']

In [65]:
#instantiating model
model_sci_note = NERModel('bert', 'allenai/scibert_scivocab_cased',labels=label,args =args, use_cuda = True)

Some weights of the model checkpoint at allenai/scibert_scivocab_cased were not used when initializing BertForTokenClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initial

In [66]:
#training model on train data and tuning hyperparameters on valid data
model_sci_note.train_model(train_data,eval_data = valid_data)

  0%|          | 0/1 [00:00<?, ?it/s]

/home/kap9623/.local/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

Running Epoch 0 of 10:   0%|          | 0/9 [00:00<?, ?it/s]

/ext3/miniconda3/lib/python3.9/site-packages/torch/optim/lr_scheduler.py:129: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "


...

Running Epoch 6 of 10:   0%|          | 0/9 [00:00<?, ?it/s]

Running Epoch 7 of 10:   0%|          | 0/9 [00:00<?, ?it/s]

Running Epoch 8 of 10:   0%|          | 0/9 [00:00<?, ?it/s]

Running Epoch 9 of 10:   0%|          | 0/9 [00:00<?, ?it/s]

(90, 0.02503834152050533)

In [67]:
# #reinstantiating NERModel using best epoch from training
# model_sci_note = NERModel('bert', 'outputs/best_model')

In [68]:
#evaluating on test data
msn_result, msn_outputs, msn_preds = model_sci_note.eval_model(test_data)

  0%|          | 0/1 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/3 [00:00<?, ?it/s]

/home/kap9623/.local/lib/python3.9/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: DRUG seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))


In [69]:
print('Model: SciBERT')
print('Evaluation metrics (Note-level split):')
msn_result

Model: SciBERT
Evaluation metrics (Note-level split):


{'eval_loss': 0.008105745364446193,
 'precision': 0.9607843137254902,
 'recall': 0.8909090909090909,
 'f1_score': 0.9245283018867925}

In [70]:
model_sci_note.predict(['As a result of this, I think it is reasonable for us in addition to having her on atenolol to stop the hydrochlorothiazide, put her on ramipril and a nitrate.'])

  0%|          | 0/1 [00:00<?, ?it/s]

Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

([[{'As': 'O'},
   {'a': 'O'},
   {'result': 'O'},
   {'of': 'O'},
   {'this,': 'O'},
   {'I': 'O'},
   {'think': 'O'},
   {'it': 'O'},
   {'is': 'O'},
   {'reasonable': 'O'},
   {'for': 'O'},
   {'us': 'O'},
   {'in': 'O'},
   {'addition': 'O'},
   {'to': 'O'},
   {'having': 'O'},
   {'her': 'O'},
   {'on': 'O'},
   {'atenolol': 'DRUG'},
   {'to': 'O'},
   {'stop': 'O'},
   {'the': 'O'},
   {'hydrochlorothiazide,': 'DRUG'},
   {'put': 'O'},
   {'her': 'O'},
   {'on': 'O'},
   {'ramipril': 'DRUG'},
   {'and': 'O'},
   {'a': 'O'},
   {'nitrate.': 'DRUG'}]],
 [[{'As': [[5.207, -6.21]]},
   {'a': [[5.09, -6.47]]},
   {'result': [[5.125, -6.363]]},
   {'of': [[5.473, -6.18]]},
   {'this,': [[5.164, -6.54], [5.184, -6.543]]},
   {'I': [[5.094, -6.42]]},
   {'think': [[5.484, -6.336]]},
   {'it': [[5.348, -6.484]]},
   {'is': [[5.137, -6.38]]},
   {'reasonable': [[5.305, -6.434]]},
   {'for': [[5.168, -6.355]]},
   {'us': [[5.273, -6.6]]},
   {'in': [[5.34, -6.28]]},
   {'addition': [[5.156,

## Patient-level Split

### Reformatting from BRAT to ConLL

In [71]:
outfile = path + '/SciBERT/output_pt_lvl.txt'
with open(outfile, 'w') as output_file:
  out_list = []
  for ann_file in ann_orig:
    with open(ann_file, 'r') as ann_in:
      txt_file = path + f'/ann_ref/{ann_file[-10:-4]}.txt'
      with open(txt_file, 'r') as txt_in:
        start_end = []
        
        #reading each line of .ann file and extracting start and end indices 
        for line in ann_in:
          entry = line.split()
          start_end.append([int(entry[2]), int(entry[3])])

        #reading each line of .txt file and extracting drug names using list of indices
        text_tokens = txt_in.read().split()
        drug_names = []
        for indicies in start_end:
          start = indicies[0]
          end = indicies[1]

          with open(txt_file, 'r') as txt_in:
            drug_names.append(txt_in.read()[indicies[0]:indicies[1]])
        
        #appending "O" or "DRUG" label to each txt token with relevant spacers
        for token in text_tokens:
          token_alnum = ''.join(ch for ch in token if ch.isalnum())
          if token_alnum in drug_names:
            if token == text_tokens[0]:
              out_list.append(f'{ann_file[-10:-7]}\t') #PATIENT LEVEL
            else:
              out_list.append(f'{np.nan}\t')
            out_list.append(f'{token}\tDRUG\n')
          else:
            if token == text_tokens[0]:
              out_list.append(f'{ann_file[-10:-7]}\t') #PATIENT LEVEL
            else:
              out_list.append(f'{np.nan}\t')
            out_list.append(f'{token}\tO\n')
        out_list.append('\n')
  out_string = ''.join(out_list)
  output_file.write(out_string)

### Reading ConLL output as dataframe

In [72]:
data = pd.read_csv(path + '/SciBERT/output_pt_lvl.txt', sep='\t', lineterminator='\n', names = ['Patient', 'Token', 'Label'])
data = data.fillna(method = 'ffill')

In [73]:
#adding token count for each token by note
token_count = []
for note in data['Patient'].unique():
  df = data.loc[data['Patient']== note]
  count = 0
  for token in df['Token']:
    token_count.append(count)
    count += 1

data['Token Count'] = token_count
data.sort_values(by = ['Patient', 'Token Count'], inplace=True)
data.reset_index(drop = True, inplace = True)

In [74]:
data.head()

,Patient,Token,Label,Token Count
0,100.0,air,O,0
1,100.0,Breeze,O,0
2,100.0,the,O,0
3,100.0,prior,O,0
4,100.0,splint,O,1


In [75]:
#label encoding Note column
le = LabelEncoder()
note_en = le.fit_transform(data['Patient'])
data['Patient'] = note_en

In [76]:
#renaming columns to match model requirements
data.rename(columns = {'Patient':'sentence_id', 'Token':'words', 'Label':'labels'}, inplace=True)

### Model Building

In [77]:
#splitting data into featues and labels
X = data[['sentence_id','words']]
y = data['labels']

#splitting into train, val, test (70-10-20)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, shuffle=False)
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size = 0.125, shuffle=False)

#building up train data and test data
train_data = pd.DataFrame({"sentence_id":X_train["sentence_id"],"words":X_train["words"],"labels":y_train})
valid_data = pd.DataFrame({"sentence_id":X_valid["sentence_id"],"words":X_valid["words"],"labels":y_valid})
test_data = pd.DataFrame({"sentence_id":X_test["sentence_id"],"words":X_test["words"],"labels":y_test})

In [78]:
#checking unique token labels
label = data["labels"].unique().tolist()
label

['O', 'DRUG']

In [79]:
#instantiating model
model_sci_pt = NERModel('bert', 'allenai/scibert_scivocab_cased',labels=label,args =args, use_cuda = True)

Some weights of the model checkpoint at allenai/scibert_scivocab_cased were not used when initializing BertForTokenClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initial

In [80]:
#training model on train data and tuning hyperparameters on valid data
model_sci_pt.train_model(train_data,eval_data = valid_data)

  0%|          | 0/1 [00:00<?, ?it/s]

/home/kap9623/.local/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

Running Epoch 0 of 10:   0%|          | 0/6 [00:00<?, ?it/s]

/ext3/miniconda3/lib/python3.9/site-packages/torch/optim/lr_scheduler.py:129: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "


...

Running Epoch 6 of 10:   0%|          | 0/6 [00:00<?, ?it/s]

Running Epoch 7 of 10:   0%|          | 0/6 [00:00<?, ?it/s]

Running Epoch 8 of 10:   0%|          | 0/6 [00:00<?, ?it/s]

Running Epoch 9 of 10:   0%|          | 0/6 [00:00<?, ?it/s]

(60, 0.04422580468193094)

In [81]:
# #reinstantiating NERModel using best epoch from training
# model_sci_pt = NERModel('bert', 'outputs/best_model')

In [82]:
#evaluating on test data
msp_result, msp_outputs, msp_preds = model_sci_pt.eval_model(test_data)

  0%|          | 0/1 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/2 [00:00<?, ?it/s]

/home/kap9623/.local/lib/python3.9/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: DRUG seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))


In [83]:
print('Model: SciBERT')
print('Evaluation metrics (Patient-level split):')
msp_result

Model: SciBERT
Evaluation metrics (Patient-level split):


{'eval_loss': 0.029693686868995428,
 'precision': 0.890625,
 'recall': 0.7702702702702703,
 'f1_score': 0.8260869565217391}

In [84]:
model_sci_pt.predict(['As a result of this, I think it is reasonable for us in addition to having her on atenolol to stop the hydrochlorothiazide, put her on ramipril and a nitrate.'])

  0%|          | 0/1 [00:00<?, ?it/s]

Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

([[{'As': 'O'},
   {'a': 'O'},
   {'result': 'O'},
   {'of': 'O'},
   {'this,': 'O'},
   {'I': 'O'},
   {'think': 'O'},
   {'it': 'O'},
   {'is': 'O'},
   {'reasonable': 'O'},
   {'for': 'O'},
   {'us': 'O'},
   {'in': 'O'},
   {'addition': 'O'},
   {'to': 'O'},
   {'having': 'O'},
   {'her': 'O'},
   {'on': 'O'},
   {'atenolol': 'DRUG'},
   {'to': 'O'},
   {'stop': 'O'},
   {'the': 'O'},
   {'hydrochlorothiazide,': 'DRUG'},
   {'put': 'O'},
   {'her': 'O'},
   {'on': 'O'},
   {'ramipril': 'DRUG'},
   {'and': 'O'},
   {'a': 'O'},
   {'nitrate.': 'DRUG'}]],
 [[{'As': [[4.223, -5.47]]},
   {'a': [[3.938, -5.543]]},
   {'result': [[4.285, -5.227]]},
   {'of': [[4.41, -4.953]]},
   {'this,': [[4.1, -5.566], [2.031, -3.053]]},
   {'I': [[3.5, -4.996]]},
   {'think': [[4.688, -5.254]]},
   {'it': [[4.66, -5.535]]},
   {'is': [[4.336, -5.42]]},
   {'reasonable': [[4.69, -5.62]]},
   {'for': [[4.09, -5.445]]},
   {'us': [[4.438, -5.695]]},
   {'in': [[4.55, -5.285]]},
   {'addition': [[4.51, -

# MODEL: ClinicalBERT

## Note-level Split

### Reformatting from BRAT to ConLL

In [85]:
outfile = path + '/ClinicalBERT/output_note_lvl.txt'
with open(outfile, 'w') as output_file:
  out_list = []
  for ann_file in ann_orig:
    with open(ann_file, 'r') as ann_in:
      txt_file = path + f'/ann_ref/{ann_file[-10:-4]}.txt'
      with open(txt_file, 'r') as txt_in:
        start_end = []
        
        #reading each line of .ann file and extracting start and end indices 
        for line in ann_in:
          entry = line.split()
          start_end.append([int(entry[2]), int(entry[3])])

        #reading each line of .txt file and extracting drug names using list of indices
        text_tokens = txt_in.read().split()
        drug_names = []
        for indicies in start_end:
          start = indicies[0]
          end = indicies[1]

          with open(txt_file, 'r') as txt_in:
            drug_names.append(txt_in.read()[indicies[0]:indicies[1]])
        
        #appending "O" or "DRUG" label to each txt token with relevant spacers
        for token in text_tokens:
          token_alnum = ''.join(ch for ch in token if ch.isalnum())
          if token_alnum in drug_names:
            if token == text_tokens[0]:
              out_list.append(f'{ann_file[-10:-4]}\t') #NOTE LEVEL
            else:
              out_list.append(f'{np.nan}\t')
            out_list.append(f'{token}\tDRUG\n')
          else:
            if token == text_tokens[0]:
              out_list.append(f'{ann_file[-10:-4]}\t') #NOTE LEVEL
            else:
              out_list.append(f'{np.nan}\t')
            out_list.append(f'{token}\tO\n')
        out_list.append('\n')
  out_string = ''.join(out_list)
  output_file.write(out_string)

### Reading ConLL output as dataframe

In [86]:
data = pd.read_csv(path + '/ClinicalBERT/output_note_lvl.txt', sep='\t', lineterminator='\n', names = ['Note', 'Token', 'Label'])
data = data.fillna(method = 'ffill')

In [87]:
#adding token count for each token by note
token_count = []
for note in data['Note'].unique():
  df = data.loc[data['Note']== note]
  count = 0
  for token in df['Token']:
    token_count.append(count)
    count += 1

data['Token Count'] = token_count
data.sort_values(by = ['Note', 'Token Count'], inplace=True)
data.reset_index(drop = True, inplace = True)

In [88]:
#label encoding Note column
le = LabelEncoder()
note_en = le.fit_transform(data['Note'])
data['Note'] = note_en

In [89]:
#renaming columns to match model requirements
data.rename(columns = {'Note':'sentence_id', 'Token':'words', 'Label':'labels'}, inplace=True)

### Model Building

In [90]:
#splitting data into featues and labels
X = data[['sentence_id','words']]
y = data['labels']

#splitting into train, val, test (70-10-20)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, shuffle=False)
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size = 0.125, shuffle=False)

#building up train data and test data
train_data = pd.DataFrame({"sentence_id":X_train["sentence_id"],"words":X_train["words"],"labels":y_train})
valid_data = pd.DataFrame({"sentence_id":X_valid["sentence_id"],"words":X_valid["words"],"labels":y_valid})
test_data = pd.DataFrame({"sentence_id":X_test["sentence_id"],"words":X_test["words"],"labels":y_test})

In [91]:
#checking unique token labels
label = data["labels"].unique().tolist()
label

['O', 'DRUG']

In [92]:
#instantiating model
model_cli_note = NERModel('bert', 'emilyalsentzer/Bio_ClinicalBERT',labels=label,args =args, use_cuda = True)

Some weights of the model checkpoint at emilyalsentzer/Bio_ClinicalBERT were not used when initializing BertForTokenClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint 

In [93]:
#training model on train data and tuning hyperparameters on valid data
model_cli_note.train_model(train_data,eval_data = valid_data)

  0%|          | 0/1 [00:00<?, ?it/s]

/home/kap9623/.local/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

Running Epoch 0 of 10:   0%|          | 0/9 [00:00<?, ?it/s]

/ext3/miniconda3/lib/python3.9/site-packages/torch/optim/lr_scheduler.py:129: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "


...

Running Epoch 6 of 10:   0%|          | 0/9 [00:00<?, ?it/s]

Running Epoch 7 of 10:   0%|          | 0/9 [00:00<?, ?it/s]

Running Epoch 8 of 10:   0%|          | 0/9 [00:00<?, ?it/s]

Running Epoch 9 of 10:   0%|          | 0/9 [00:00<?, ?it/s]

(90, 0.043555733493687715)

In [94]:
# #reinstantiating NERModel using best epoch from training
# model_cli_note = NERModel('bert', 'outputs/best_model')

In [95]:
#evaluating on test data
mcn_result, mcn_outputs, mcn_preds = model_cli_note.eval_model(test_data)

  0%|          | 0/1 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/3 [00:00<?, ?it/s]

/home/kap9623/.local/lib/python3.9/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: DRUG seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))


In [96]:
print('Model: ClinicalBERT')
print('Evaluation metrics (Note-level split):')
mcn_result

Model: ClinicalBERT
Evaluation metrics (Note-level split):


{'eval_loss': 0.005606188516442974,
 'precision': 0.8936170212765957,
 'recall': 0.8936170212765957,
 'f1_score': 0.8936170212765957}

In [97]:
model_cli_note.predict(['As a result of this, I think it is reasonable for us in addition to having her on atenolol to stop the hydrochlorothiazide, put her on ramipril and a nitrate.'])

  0%|          | 0/1 [00:00<?, ?it/s]

Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

([[{'As': 'O'},
   {'a': 'O'},
   {'result': 'O'},
   {'of': 'O'},
   {'this,': 'O'},
   {'I': 'O'},
   {'think': 'O'},
   {'it': 'O'},
   {'is': 'O'},
   {'reasonable': 'O'},
   {'for': 'O'},
   {'us': 'O'},
   {'in': 'O'},
   {'addition': 'O'},
   {'to': 'O'},
   {'having': 'O'},
   {'her': 'O'},
   {'on': 'O'},
   {'atenolol': 'DRUG'},
   {'to': 'O'},
   {'stop': 'O'},
   {'the': 'O'},
   {'hydrochlorothiazide,': 'DRUG'},
   {'put': 'O'},
   {'her': 'O'},
   {'on': 'O'},
   {'ramipril': 'DRUG'},
   {'and': 'O'},
   {'a': 'O'},
   {'nitrate.': 'DRUG'}]],
 [[{'As': [[5.113, -5.25]]},
   {'a': [[5.03, -5.125]]},
   {'result': [[5.023, -4.797]]},
   {'of': [[4.977, -4.78]]},
   {'this,': [[4.965, -4.598], [4.91, -4.535]]},
   {'I': [[5.375, -4.77]]},
   {'think': [[5.312, -4.543]]},
   {'it': [[5.33, -4.754]]},
   {'is': [[5.25, -4.78]]},
   {'reasonable': [[5.22, -4.746]]},
   {'for': [[5.07, -4.52]]},
   {'us': [[5.13, -4.805]]},
   {'in': [[5.38, -4.984]]},
   {'addition': [[5.15, -4

## Patient-level Split

### Reformatting from BRAT to ConLL

In [98]:
outfile = path + '/ClinicalBERT/output_pt_lvl.txt'
with open(outfile, 'w') as output_file:
  out_list = []
  for ann_file in ann_orig:
    with open(ann_file, 'r') as ann_in:
      txt_file = path + f'/ann_ref/{ann_file[-10:-4]}.txt'
      with open(txt_file, 'r') as txt_in:
        start_end = []
        
        #reading each line of .ann file and extracting start and end indices 
        for line in ann_in:
          entry = line.split()
          start_end.append([int(entry[2]), int(entry[3])])

        #reading each line of .txt file and extracting drug names using list of indices
        text_tokens = txt_in.read().split()
        drug_names = []
        for indicies in start_end:
          start = indicies[0]
          end = indicies[1]

          with open(txt_file, 'r') as txt_in:
            drug_names.append(txt_in.read()[indicies[0]:indicies[1]])
        
        #appending "O" or "DRUG" label to each txt token with relevant spacers
        for token in text_tokens:
          token_alnum = ''.join(ch for ch in token if ch.isalnum())
          if token_alnum in drug_names:
            if token == text_tokens[0]:
              out_list.append(f'{ann_file[-10:-7]}\t') #PATIENT LEVEL
            else:
              out_list.append(f'{np.nan}\t')
            out_list.append(f'{token}\tDRUG\n')
          else:
            if token == text_tokens[0]:
              out_list.append(f'{ann_file[-10:-7]}\t') #PATIENT LEVEL
            else:
              out_list.append(f'{np.nan}\t')
            out_list.append(f'{token}\tO\n')
        out_list.append('\n')
  out_string = ''.join(out_list)
  output_file.write(out_string)

### Reading ConLL output as dataframe

In [99]:
data = pd.read_csv(path + '/ClinicalBERT/output_pt_lvl.txt', sep='\t', lineterminator='\n', names = ['Patient', 'Token', 'Label'])
data = data.fillna(method = 'ffill')

In [100]:
#adding token count for each token by note
token_count = []
for note in data['Patient'].unique():
  df = data.loc[data['Patient']== note]
  count = 0
  for token in df['Token']:
    token_count.append(count)
    count += 1

data['Token Count'] = token_count
data.sort_values(by = ['Patient', 'Token Count'], inplace=True)
data.reset_index(drop = True, inplace = True)

In [101]:
data.head()

,Patient,Token,Label,Token Count
0,100.0,air,O,0
1,100.0,Breeze,O,0
2,100.0,the,O,0
3,100.0,prior,O,0
4,100.0,splint,O,1


In [102]:
#label encoding Note column
le = LabelEncoder()
note_en = le.fit_transform(data['Patient'])
data['Patient'] = note_en

In [103]:
#renaming columns to match model requirements
data.rename(columns = {'Patient':'sentence_id', 'Token':'words', 'Label':'labels'}, inplace=True)

### Model Building

In [104]:
#splitting data into featues and labels
X = data[['sentence_id','words']]
y = data['labels']

#splitting into train, val, test (70-10-20)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, shuffle=False)
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size = 0.125, shuffle=False)

#building up train data and test data
train_data = pd.DataFrame({"sentence_id":X_train["sentence_id"],"words":X_train["words"],"labels":y_train})
valid_data = pd.DataFrame({"sentence_id":X_valid["sentence_id"],"words":X_valid["words"],"labels":y_valid})
test_data = pd.DataFrame({"sentence_id":X_test["sentence_id"],"words":X_test["words"],"labels":y_test})

In [105]:
#checking unique token labels
label = data["labels"].unique().tolist()
label

['O', 'DRUG']

In [106]:
#instantiating model
model_cli_pt = NERModel('bert', 'allenai/scibert_scivocab_cased',labels=label,args =args, use_cuda = True)

Some weights of the model checkpoint at allenai/scibert_scivocab_cased were not used when initializing BertForTokenClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initial

In [107]:
#training model on train data and tuning hyperparameters on valid data
model_cli_pt.train_model(train_data,eval_data = valid_data)

  0%|          | 0/1 [00:00<?, ?it/s]

/home/kap9623/.local/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

Running Epoch 0 of 10:   0%|          | 0/6 [00:00<?, ?it/s]

/ext3/miniconda3/lib/python3.9/site-packages/torch/optim/lr_scheduler.py:129: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "


...

Running Epoch 6 of 10:   0%|          | 0/6 [00:00<?, ?it/s]

Running Epoch 7 of 10:   0%|          | 0/6 [00:00<?, ?it/s]

Running Epoch 8 of 10:   0%|          | 0/6 [00:00<?, ?it/s]

Running Epoch 9 of 10:   0%|          | 0/6 [00:00<?, ?it/s]

(60, 0.04422580468193094)

In [108]:
# #reinstantiating NERModel using best epoch from training
# model_cli_pt = NERModel('bert', 'outputs/best_model')

In [109]:
#evaluating on test data
mcp_result, mcp_outputs, mcp_preds = model_cli_pt.eval_model(test_data)

  0%|          | 0/1 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/2 [00:00<?, ?it/s]

/home/kap9623/.local/lib/python3.9/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: DRUG seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))


In [110]:
print('Model: ClinicalBERT')
print('Evaluation metrics (Patient-level split):')
mcp_result

Model: ClinicalBERT
Evaluation metrics (Patient-level split):


{'eval_loss': 0.029693686868995428,
 'precision': 0.890625,
 'recall': 0.7702702702702703,
 'f1_score': 0.8260869565217391}

In [111]:
model_cli_pt.predict(['As a result of this, I think it is reasonable for us in addition to having her on atenolol to stop the hydrochlorothiazide, put her on ramipril and a nitrate.'])

  0%|          | 0/1 [00:00<?, ?it/s]

Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

([[{'As': 'O'},
   {'a': 'O'},
   {'result': 'O'},
   {'of': 'O'},
   {'this,': 'O'},
   {'I': 'O'},
   {'think': 'O'},
   {'it': 'O'},
   {'is': 'O'},
   {'reasonable': 'O'},
   {'for': 'O'},
   {'us': 'O'},
   {'in': 'O'},
   {'addition': 'O'},
   {'to': 'O'},
   {'having': 'O'},
   {'her': 'O'},
   {'on': 'O'},
   {'atenolol': 'DRUG'},
   {'to': 'O'},
   {'stop': 'O'},
   {'the': 'O'},
   {'hydrochlorothiazide,': 'DRUG'},
   {'put': 'O'},
   {'her': 'O'},
   {'on': 'O'},
   {'ramipril': 'DRUG'},
   {'and': 'O'},
   {'a': 'O'},
   {'nitrate.': 'DRUG'}]],
 [[{'As': [[4.223, -5.47]]},
   {'a': [[3.938, -5.543]]},
   {'result': [[4.285, -5.227]]},
   {'of': [[4.41, -4.953]]},
   {'this,': [[4.1, -5.566], [2.031, -3.053]]},
   {'I': [[3.5, -4.996]]},
   {'think': [[4.688, -5.254]]},
   {'it': [[4.66, -5.535]]},
   {'is': [[4.336, -5.42]]},
   {'reasonable': [[4.69, -5.62]]},
   {'for': [[4.09, -5.445]]},
   {'us': [[4.438, -5.695]]},
   {'in': [[4.55, -5.285]]},
   {'addition': [[4.51, -